# Beispiel: Lademanagement eines Speichers für Eigenbedarfsoptimierung

#### Imports

In [ ]:
%pip install pandas
%pip install numpy
%pip install openpyxl
import pandas as pd
import numpy as np

#### Daten lesen

In [ ]:
pfad = "PV+Speicher_EV-Opt.xlsx"
data = pd.read_excel(pfad,sheet_name="Aufgabe", names=["t","P_e_rel","P_v"], 
                     usecols="A:C", skiprows=28, parse_dates=['t'])
data=data.set_index('t')
data.P_e_rel = data.P_e_rel/1000

#### Parameter

P_Nenn ... Leistung unter Standard Testbedingungen [W] (oft ugs. [Wp])

C ... Speicherkapazität [Wh]

eta ... Speicherwirkungsgrad 

selb ... Selbstentladung pro Monat 

dt ... Zeitschritt [h]


In [ ]:
P_Nenn= 6000
C = 5000
eta = 0.9
selb = 0.05
dt = 1

# Erzeugte Leitstung berechnen
data["P_e"] = data.P_e_rel*P_Nenn

# Differenz erzeugte und verbrauchte Leistung
data["P_diff"] = data.P_v-data.P_e

In [ ]:
#data = data.resample('4h').mean()
#dt = 4

In [ ]:
data[["P_v","P_e"]].plot(ylabel = "P [W]")

In [ ]:
data[["P_v","P_e"]].loc["2019-6-1":"2019-6-7"].plot(ylabel = "P [W]")

In [ ]:
data[["P_v","P_e"]].loc["2019-6-6"].plot(ylabel = "P [W]")

In [ ]:
data[["P_v","P_e","P_diff"]].describe()

#### Simulation

In [ ]:
#Selbstentladung pro dt berechnen
P_selb_max = C * selb * 1/30/24 * dt 
#maximale Ladeleistung/Entladeleistung pro dt
P_laden_max = C / dt
#Initialisieren - Anfangswert
Q_alt = 0 
Q_arr = []
P_ein_arr = []
P_aus_arr = []
P_abw_netz_arr = []
P_abw_ein_arr = []

for t in data.index:
    P_selb = np.min([P_selb_max, data.P_e.loc[t]])
    P_soll = data.P_v.loc[t] - data.P_e.loc[t]
    if P_soll <= 0:
        P_ein = np.min([-P_soll, P_laden_max, (C-Q_alt+P_selb*dt)/(eta*dt)])
        P_aus = 0
        P_abw_ein = - P_soll - P_ein
        P_abw_netz = 0
        Q_neu = Q_alt + eta * P_ein * dt
    else:
        P_aus = np.min([P_soll, P_laden_max, Q_alt/dt])
        P_ein = 0
        P_abw_netz = P_soll - P_aus
        P_abw_ein = 0
        Q_neu = Q_alt - P_aus * dt
    Q_neu = np.max([Q_neu - P_selb * dt, 0])
    Q_arr.append(Q_neu)
    P_ein_arr.append(P_ein)
    P_aus_arr.append(P_aus)
    P_abw_netz_arr.append(P_abw_netz)
    P_abw_ein_arr.append(P_abw_ein)

    Q_alt = Q_neu
    


results = pd.DataFrame({"Q":Q_arr, "P_ein":P_ein_arr, "P_aus":P_aus_arr, 
                        "P_netz":P_abw_netz_arr, "P_einsp":P_abw_ein_arr}, index = data.index)
results["P_direkt"] = data[["P_v","P_e"]].min(axis=1)

#### Validierungs Check

In [ ]:
results.Q.loc["2019-6-21":"2019-6-30"].plot(ylabel = "Q [Wh]")

In [ ]:
results[["P_ein", "P_aus", "P_netz", "P_einsp", "P_direkt"]].loc["2019-6-21":"2019-6-30"].plot(ylabel = "P [W]")

In [ ]:
ax = results[["P_ein", "P_aus", "P_netz", "P_einsp", "P_direkt"]].resample('m').mean().plot()
data[["P_e", "P_v"]].resample('m').mean().plot(ax=ax, ylabel="P[W]", title = "Monatsmittelwerte")

#### Monatssummen in kWh

In [ ]:
results[["P_ein", "P_aus", "P_netz", "P_einsp", "P_direkt"]].resample('m').sum().multiply(dt/1000)

In [ ]:
data[["P_v", "P_e"]].resample('m').sum().multiply(dt/1000)

In [ ]:
results[["P_ein", "P_aus", "P_netz", "P_einsp", "P_direkt"]].sum().multiply(dt/1000)

In [ ]:
data[["P_v", "P_e"]].sum().multiply(dt/1000)

#### KPIs

Eigenverbrauch in kWh

In [ ]:
ev = (data.P_v.sum()-results.P_netz.sum())*dt/1000
ev

Eigenverbrauchsquote EVQ

In [ ]:
ev/(data.P_e.sum()*dt/1000)

Solarer Deckungsgrad SDQ

In [ ]:
ev/(data.P_v.sum()*dt/1000)

Ladezyklen

In [ ]:
results.P_aus.sum()*dt/C